In [55]:
import pandas as pd
import psycopg2 as pg
import sqlalchemy as sq

In [53]:
pd.options.display.max_columns = 30
!wget https://data.baltimorecity.gov/api/views/n4ma-fj3m/rows.csv?accessType=DOWNLOAD -O Parking_Citations.csv

In [60]:
df = pd.read_csv('Parking_Citations.csv')

/Users/weizheng/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
# convert to date type
df.ImportDate = df.ImportDate.str.replace(' \+0000', '')
df.NoticeDate = pd.to_datetime(df.NoticeDate, format="%m/%d/%Y")
df.PenaltyDate = pd.to_datetime(df.PenaltyDate, format="%m/%d/%Y")
df.ImportDate = pd.to_datetime(df.ImportDate, format="%m/%d/%Y %H:%M:%S %p")
df.ViolDate = pd.to_datetime(df.ViolDate, format="%m/%d/%Y %H:%M:%S %p")
# split Location
df['Latitude'] = df.Location.str.split(',').str.get(0).str.replace('\(','').map(lambda s: float(s))
df['Longitude'] = df.Location.str.split(',').str.get(1).str.replace('\)','').map(lambda s: float(s))
# remove $ sign
df[['ViolFine', 'Balance','OpenFine','OpenPenalty']] = df[['ViolFine', 'Balance','OpenFine','OpenPenalty']].applymap(lambda s: float(str(s).replace('$', '')))


In [62]:
df.dtypes

Citation                    int64
Tag                        object
ExpMM                      object
ExpYY                     float64
State                      object
Make                       object
Address                    object
ViolCode                  float64
Description                object
ViolFine                  float64
ViolDate           datetime64[ns]
Balance                   float64
PenaltyDate        datetime64[ns]
OpenFine                  float64
OpenPenalty               float64
NoticeDate         datetime64[ns]
ImportDate         datetime64[ns]
Neighborhood               object
PoliceDistrict             object
CouncilDistrict           float64
Location                   object
Latitude                  float64
Longitude                 float64
dtype: object

In [58]:
df.head(1)

,Citation,Tag,ExpMM,ExpYY,State,Make,Address,ViolCode,Description,ViolFine,...,OpenFine,OpenPenalty,NoticeDate,ImportDate,Neighborhood,PoliceDistrict,CouncilDistrict,Location,Latitude,Longitude
0,22301139,T879530T,11,13.0,MD,LEXUS,800 CAROLINE ST,20.0,Passenger Loading Zone,32.0,...,32.0,0.0,NaT,2013-11-13 04:03:00,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
%load_ext sql
!echo 'redspot' | sudo -S service postgresql restart
!createdb -U dbuser project3
%sql postgresql://dbuser@localhost:5432/project3
engine = sq.create_engine("postgresql+psycopg2://dbuser@localhost:5432/project3")
df_fact.to_sql('citation', engine, if_exists='replace', index=False)

Timestamp('2016-11-25 10:13:00')